In [20]:
import scipy.io as spio
import numpy as np
import math
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from collections import Counter


In [21]:
def readMat(dataPath):
    
    
    #readMat: Reads the .mat for mad river data files placed at data path
    #INPUT
    #dataPath == relative location of the data path to the code file, or absolute
    #
    #OUTPUT
    #still being worked on
    #eventToClass== np array of all classes belonging to all 616events. value at 0 index is class for 0th event.
    #sedimentData_events == list containing array for sediment data for each event
    #
    #
    #events == list containing 2-D array for each event. 1st column is sediment and 2nd is stream flow. Ideally we will be using this
    #maxEventLen == Longest event in terms of timesteps
    
    #sample call: eventToClass, myEvents, maxEventLen,streamFlow_Data,sedimentData_events = readMat('..\data\')
    
    #Programmer: Ali Javed
    #Date last modified: 27 Feb 2018
    #modified by: Ali Javed
    #Comments: Initial version.
    
    
    ##############################################################################################################
        
    
    
    
    classMat = spio.loadmat(dataPath + 'allMadSitesStormHystClassKey.mat', squeeze_me=True)
    dataMat = spio.loadmat(dataPath + 'allMadSitesEventTimeSeries.mat', squeeze_me=True)
    
    eventToClass = classMat['stormHystClass'][:,3] #index 3 refers to class of 3rd event. Event number start from 0
    eventToClass = eventToClass.astype(int) # we do not need float classes




    #gather 626 events
    events = []
    sedimentData_events = []
    streamFlowData_events = []
    counter = 0
    maxEventLen = -1 #need this as fixed size input to keras RNN
    
    streamFlow = 1
    suspendedSedimentConcentration = 2
    
    for event in range(0,len(dataMat['dataTSOut'])):
    
    
        #not reading datetime and rainfall data for now
        #event_dataTime = np.zeros((len(dataMat['dataTSOut'][event][streamFlow]))) #can not extract datetime so setting it to one, for out purpose it does not matter anyways
        #event_rainFall = np.zeros((len(dataMat['dataTSOut'][event][streamFlow])))
                             
        event_streamflow = dataMat['dataTSOut'][event][streamFlow]
        event_suspendedSedimentConcentration = dataMat['dataTSOut'][event][suspendedSedimentConcentration]
    
        
        eventArray = np.column_stack((event_streamflow,event_suspendedSedimentConcentration))
        
        
        events.append(eventArray)
        sedimentData_events.append(event_streamflow)
        streamFlowData_events.append(event_suspendedSedimentConcentration)
    
        if len(event_streamflow)> maxEventLen:
            maxEventLen = len(event_streamflow)
    
    
        
        ##############################################################################################################
        #for classification based only on rain and sediment... i can not figure out how to give 2d input to RNN
        
        
        #classVector = np.repeat(eventToClass[event], len(event_streamflow))
        #print(np.shape(classVector))
        #print(np.shape(suspendedSedimentConcentration))
        #streamFlow_Data = np.column_stack((event_streamflow,classVector))
        #suspendedSedimentConcentration_Data = np.column_stack((event_suspendedSedimentConcentration,classVector))
        
    return eventToClass, events, maxEventLen, streamFlowData_events, sedimentData_events
    
    
 ##############################################################################################################
       

 



 
    


In [22]:
dataPath = '../data/'

#eventToClass as an array of len(events) with each index telling the class of event
#myEvents is a list containing 2-d arrays for all events. 0 column is the stream flow, 1 column is sediment concentration
eventToClass, myEvents, maxEventLen,streamFlow_Data,sedimentData_events = readMat(dataPath)




In [23]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical
# fix random seed for reproducibility
numpy.random.seed(7)



In [25]:
#train test split startified not being done for now

#preprocess data to required format, padding to make all sequence data same lenght
X_data = sequence.pad_sequences(sedimentData_events,dtype='float')
#create one hot representation for class values
y_data = to_categorical(eventToClass, num_classes=None)

In [40]:
#Test/Train Split

sampler = StratifiedKFold(n_splits=4, shuffle=False, random_state=None)


#sampler.split(x= X_data,y=eventToClass)

for train_index, test_index in sampler.split(X_data, eventToClass):
    X_train, X_test = X_data[train_index], X_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

In [ ]:
#############################################################################
#DECLARE PARAMETERS FOR NN
#these parameters are the architecture of the RNN. Still have to do more on this
embedding_vecor_length = 32  #each event is represented using a 32 length vector. 
len_input = 700  #i do not know what purpose these vectors do


###############################################################################

In [42]:
###############################################################################
#CREATE SIMPLE RNN


# create the model

model = Sequential()

model.add(Embedding(len_input, embedding_vecor_length, input_length=maxEventLen))
#add the recurrent LSTM layer of 100 nodes
model.add(LSTM(100))

#out put layer with 16 nodes, one hot representation
model.add(Dense(16, activation='relu'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

#test and train both on same data because did not split for now.
model.fit(X_train, y_train, epochs=3, batch_size=25)
Loss,Acc =model.evaluate(x = X_test, y = y_test)


print("RNN accuracy is ",Acc)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 313, 32)           22400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 16)                1616      
Total params: 77,216
Trainable params: 77,216
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
475/475 [==============================] - 20s 42ms/step - loss: 2.7236 - acc: 0.1874
Epoch 2/3
475/475 [==============================] - 19s 40ms/step - loss: 2.4640 - acc: 0.2000
Epoch 3/3
151/151 [==============================] - 1s 4ms/step


[2.4016843742092715, 0.20529801373844905]

In [32]:
Counter(y_test)

Counter({1: 5,
         2: 2,
         3: 3,
         4: 8,
         5: 27,
         6: 14,
         7: 8,
         8: 14,
         9: 2,
         10: 6,
         11: 6,
         12: 11,
         13: 8,
         14: 1,
         15: 11})

In [34]:
Counter(y_train)

Counter({1: 19,
         2: 10,
         3: 5,
         4: 39,
         5: 97,
         6: 68,
         7: 48,
         8: 78,
         9: 10,
         10: 17,
         11: 18,
         12: 40,
         13: 25,
         14: 7,
         15: 19})

In [41]:
len(y_train)

500